# Steps for the first run

- Run `conda install -c conda-forge r-plyr r-dplyr r-data.table r-bit64 r-geosphere r-tidyverse r-maps r-scales r-stringr r-ggplot2` using command line and restart the R kernel of this notebook
- Run the next `install.packages(c("feather", "geosphere")) cell` - required for some reason
- For some reason the notebook does not work beyond cell 16. Use `nbconvert` in command line, or nbconvertR package in R to convert this notebook to a .r file. Either use R prompt or use `Rscript` command to run the generated .r file. To successfully execute nbconvert in R:
    - Install package nbconvertR using `conda install -c r r-nbconvertr`
    - `library(nbconvertR)`
    - `nbconvert("postprocess_new_adsb.ipynb", "script")`
    - This will produce postprocess_new_adsb.r that can either be sourced from R prompt using `source` or from command line using `Rscript`

In [ ]:
#install.packages(c("feather", "geosphere"))

In [ ]:
# library(nbconvertR)
# nbconvert("01-label_aircraft_id.ipynb", "script")

# Data dictionary

| Variable | Description |
|----------|-------------|
|    V1    | Unknown timestamp |
|    V2    | Seems to change for each aircraft during exit-entry |
|    V3    | Looks like a proxy for aircraft |
|    V4    | |
|    V5    | |
|    V6    | Heading direction in degrees, ranges from 1 to 360 |
|    V7    | (Barometric?) Altitude in ft |
|    V8    | (Estimated?) Ground speed in kts? |
|    V9    | |
|    V10   | Call-sign: not a good proxy for aircraft as one might expect |
|    V11   | |
|    V12   | |
|    V13   | Useful timestamp? |
|    V14   | Source airport code |
|    V15   | Destination airport code |
|    V16   | Another proxy for aircraft? |
|    V17   | Flag indicating if the aircraft is at an airport? |
|    V18   | Instantaneous climb rate (ft/s)? (jittery) |
|    V19   | Another proxy for aircraft? |
|  V16_V19 | Looks like a good proxy for aircraft ID. Can be used to refine V3 further |

# Definitions

| Term | Definition |
|------|------------|
| Maximum time between two samples for the aircraft to be declared "expired" | 2 hours |
| Time from exit to re-entry as a new aircraft | 18 mins |
| Minimum time to remain on ground to be labeled as a new aircraft | 18 mins |
| Absolute latitude difference around airport for an aircraft to be labeled 'landed' | 0.1&deg; |
| Absolute longitude difference around airport for an aircraft to be labeled 'landed' | 0.1&deg; |

# Initialization

In [1]:
lower_lat <- 35
upper_lat <- 45
lower_lon <- -78
upper_lon <- -68
lat_range <- 0.4
lon_range <- 0.5
ts_diff_ground <- 1080
ts_diff_exit_enter <- 1080
ts_diff_expired <- 7200
airport_lat_half_range <- airport_lon_half_range <- 0.1

# Constraints

In [4]:
v1_ground <- 50 # Minimum speed for takeoff/landing
landed_altitude <- 1000 # may not need this because of V17
surely_grounded_speed <- 10 # Aircraft is guaranteed to be on the ground at this speed
alt_100_99_perc_ratio_max <- 1.05 # Ratio of max altitude to 99%ile altitude should be <= 1.05, otherwise drop the sample (not the entire aircraft data)
altitude_ceiling <- 48000

# Loading packages and utilities

In [2]:
suppressMessages({
    library(plyr)
    library(dplyr)
    library(data.table)
    library(bit64)
    library(geosphere)
    library(feather)
    # library(rnaturalearth)
})

In [3]:
process_ac_data <- function(ac_data) {
  ac_data <- ac_data %>% rename(
      azimuth = V6, source = V14, destination = V15, altitude = V7,
      ground_speed = V8, aircraft = V3, ts = V13, climb_rate = V18,
      # ground_speed = V8, aircraft = V10, ts = V1, climb_rate = V18,
      ground_flag = V17, lat = V4, lon = V5, aircraft_type = V11)

  # New: Ground flag and altitude update to avoid discrepancies
  ac_data$ground_flag[(ac_data$altitude == 0) & (ac_data$ground_speed <= v1_ground)] <- 1
  ac_data$altitude[ac_data$ground_flag == 1] <- 0

  ac_data$altitude <- 100 * ceiling(ac_data$altitude/100) # Ceiling altitude to nearest 100
  ac_data <- ac_data[ac_data$V10 != "F-EST"] # Removing estimated positions
  setorder(ac_data, aircraft, ts)
  ac_data <- ac_data[, .SD[1], by = list(aircraft, ts)]

  ac_data$ts_readable <- as.POSIXct.numeric(as.numeric(ac_data$ts), origin="1970-01-01")

  # Removing anomalous aircraft
  ac_data$quadrant <- ceiling(ac_data$azimuth/90)
  ac_data <- ac_data[!grepl(ac_data$aircraft, pattern = "^FAA"), ]
  # ac_to_remove <- data.frame(aircraft = unique(ac_data$aircraft[ac_data$altitude >= 30000 & ac_data$ground_speed < 100]))

  # ac_data$V16_V19 <- paste0(ac_data$V16, "_", ac_data$V19) # Concatenates both available IDs
  return(ac_data)
}

In [5]:
csv_files <- list.files(pattern = "^ads-b.*\\.csv$")
print(csv_files)
dt_lst <- list(csv_files)

[1] "ads-b_with_id.csv"


In [6]:
if(length(csv_files) > 0) {
  for(i in 1:length(csv_files)) {
    csv_file <- csv_files[i]
    dt_lst[[i]] <- process_ac_data(fread(csv_file, header = FALSE))
  }
}
ac_data <- rbindlist(dt_lst, fill = TRUE)
rm(dt_lst)
# unique(ac_data, by = c("aircraft", "ts")) # Not good because "aircraft" is not strictly unique
gc(verbose = FALSE)

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,1324525,70.8,2439237,130.3,1362794,72.8
Vcells,108521717,828.0,342253930,2611.2,427475044,3261.4


In [7]:
# ac_data <- fread("ads-b_with_id.csv")
# ac_data <- rbind(ac_data, fread("../../adsb_fetch/ads-b_exta_with_id_no_est.csv"))

setorder(ac_data, cols = "aircraft", "ts")

In [8]:
head(ac_data)

aircraft,ts,V1,V2,lat,lon,azimuth,altitude,ground_speed,V9,⋯,destination,V16,ground_flag,climb_rate,V19,V20,V21,ts_readable,quadrant,V16_V19
<chr>,<int>,<dbl>,<chr>,<dbl>,<dbl>,<int>,<dbl>,<int>,<lgl>,⋯,<chr>,<chr>,<dbl>,<int>,<chr>,<int>,<chr>,<dttm>,<dbl>,<chr>
01008C,1686241896,1686241897,30a21562,42.252,-69.212,0,41700,0,NA,⋯,,,0,192,,0,EGY,2023-06-08 16:31:36,0,T-MLAT2__
01008C,1686241916,1686241922,30a21562,42.225,-69.276,241,24000,240,NA,⋯,,,0,192,,0,EGY,2023-06-08 16:31:56,3,T-MLAT2__
01008C,1686241928,1686241931,30a21562,42.216,-69.298,241,24000,240,NA,⋯,,,0,-6400,,0,EGY,2023-06-08 16:32:08,3,T-MLAT2__
01008C,1686241940,1686241939,30a21562,42.208,-69.319,242,24000,244,NA,⋯,,,0,0,,0,EGY,2023-06-08 16:32:20,3,T-MLAT2__
01008C,1686241948,1686241948,30a21562,42.203,-69.332,243,24000,246,NA,⋯,,,0,0,,0,EGY,2023-06-08 16:32:28,3,T-MLAT2__
01008C,1686241957,1686241956,30a21562,42.201,-69.335,243,41700,246,NA,⋯,,,0,0,,0,EGY,2023-06-08 16:32:37,3,T-MLAT2__


In [9]:
ac_data[, c("last_lat", "last_lon", "last_azimuth", "last_altitude",
            "last_ts", "last_source", "last_destination", "last_V9",
            #"last_ground_flag", "last_V16_V19", "last_quadrant",
            "last_ground_flag", "last_quadrant",
            "last_ground_speed", "last_V2",
            "next_lat", "next_lon", "next_azimuth", "next_altitude",
            "next_ts", "next_source", "next_destination", "next_V9",
            #"next_ground_flag", "next_V16_V19", "next_quadrant",
            "next_ground_flag", "next_quadrant",
            "next_ground_speed", "next_V2",
            "uniq_source",
            "uniq_destination",
            "max_ac_alt") :=
                list(c(lat[1], lat[-.N]), c(lon[1], lon[-.N]), c(azimuth[1], azimuth[-.N]), c(altitude[1], altitude[-.N]),
                     c(ts[1], ts[-.N]), c(NA, source[-.N]), c(NA, destination[-.N]), c(NA, V9[-.N]),
                     #c(ground_flag[1], ground_flag[-.N]), c(V16_V19[1], V16_V19[-.N]), c(quadrant[1], quadrant[-.N]),
                     c(ground_flag[1], ground_flag[-.N]), c(quadrant[1], quadrant[-.N]),
                     c(ground_speed[1], ground_speed[-.N]), c(V2[1], V2[-.N]),
                     c(lat[-.1], NA), c(lon[-1], NA), c(azimuth[-.1], NA), c(altitude[-1], NA),
                     c(ts[-1], NA), c(source[-1], NA), c(destination[-1], NA), c(V9[-1], NA),
                     #c(ground_flag[-1], NA), c(V16_V19[-1], NA), c(quadrant[-1], NA),
                     c(ground_flag[-1], NA), c(quadrant[-1], NA),
                     c(ground_speed[-1], NA), c(V2[-1], NA),
                     paste0(sort(unique(source)), collapse = " "),
                     paste0(sort(unique(destination)), collapse = " "),
                     max(altitude)), by = "aircraft"]
ac_data <- ac_data[ac_data$max_ac_alt > 0, ] # Dropping aircraft (not IDs) that have max altitude = 0
ac_data$last_ts_diff <- ac_data$ts - ac_data$last_ts
ac_data$next_ts_diff <- ac_data$next_ts - ac_data$ts
ac_data$destination <- toupper(ac_data$destination)
ac_data$est_climb_rate <- 60 * (as.numeric(ac_data$altitude) - as.numeric(ac_data$last_altitude))/as.numeric(ac_data$last_ts_diff) # ft/min
ac_data$climb_rate_ratio <- 1

# Filling climb_rate_ratio if est_climb_rate!=0 and est_climb_rate is not null and last_ts_diff < 20 mins
condition <- ac_data$est_climb_rate != 0 & !is.na(ac_data$est_climb_rate) & ac_data$last_ts_diff < 1200
ac_data$climb_rate_ratio[condition] <- ac_data$climb_rate[condition]/ac_data$est_climb_rate[condition]

# Change in heading quadrant
ac_data$quadrant_change <- ac_data$quadrant != ac_data$last_quadrant

# Reading airports data and checking

In [10]:
airports <- read.csv("GlobalAirportDatabase.txt", header = F, stringsAsFactors = F, sep = ":")
airports <- airports %>% dplyr::rename(airport = V2, lat = V15, lon = V16)
airports$airport <- toupper(airports$airport)
airports <- airports[(airports$airport != "") & (airports$airport != "N/A"), ]

# 511 of the 563 destinations can be matched

airports <- airports[data.table::between(airports$lat, lower = lower_lat, upper = upper_lat) &
                       data.table::between(airports$lon, lower = lower_lon, upper = upper_lon), ]
rownames(airports) <- airports$V1 <- NULL
head(airports$airport, 20)

[1] "YGK" "YTR" "GWW" "ACK" "ACY" "ADW" "ALB" "APG" "ART" "AUG" "BDL" "BDR"
[13] "BED" "BGR" "BOS" "BTV" "BWI" "CEF" "DCA" "DOV"

In [11]:
table(ac_data$altitude[ac_data$ground_flag == 1])


      0 
1221858 

# Defining landing and entry into sector

In [12]:
ac_data$touchdown <- (ac_data$last_ground_flag == 0) &
  (ac_data$ground_flag == 1)
ac_data[, "last_touchdown" := c(F, touchdown[-.N]), by = aircraft]
ac_data$touchdown <- ac_data$last_touchdown
ac_data$last_touchdown <- NULL

In [13]:
ac_data[, "touchdown_id" := cumsum(touchdown), by = aircraft]
ac_data[, "max_touchdown" := max(touchdown_id), by = aircraft]
ac_data[, max_altitude_touchdown_id := max(altitude), by = list(aircraft, touchdown_id)]
print(sum(ac_data$touchdown & (ac_data$max_altitude_touchdown_id < 0.1 * ac_data$max_ac_alt)))
ac_data$touchdown[ac_data$touchdown & (ac_data$max_altitude_touchdown_id < 0.1 * ac_data$max_ac_alt)] <- F
ac_data[, "touchdown_id" := cumsum(touchdown), by = aircraft]
ac_data[, "max_touchdown" := max(touchdown_id), by = aircraft]
ac_data[, max_altitude_touchdown_id := max(altitude), by = list(aircraft, touchdown_id)]

[1] 2497


Consecutive landings within 10 mins is not possible. These can be considered as missed approaches, therefore only the final landing is considered

In [15]:
false_idx <- c()
uniq_ac <- unique(ac_data$aircraft[ac_data$touchdown & ac_data$max_touchdown > 1])
print(length(uniq_ac))

[1] 1086


Something's wrong with the R kernel in a notebook. So, the following lines were executed in R shell.

In [16]:
j <- 0
for(i in 1:length(uniq_ac)) {
    if(((i * 100) / length(uniq_ac)) >= j) {
        print(paste0(j, "% complete"))
        j <- j + 10
    }
    ac <- uniq_ac[i]
    ac_df <- ac_data[ac_data$aircraft == ac & ac_data$touchdown, ]
    time_between_touchdowns <- as.numeric(difftime(ac_df$ts_readable[-1], ac_df$ts_readable[-nrow(ac_df)], units = "s"))
    if(any(time_between_touchdowns < 600)) {
        idx1 <- which(time_between_touchdowns < 600)
        ac_data_idx <- which(ac_data$aircraft == ac & ac_data$touchdown)
        false_idx <- c(false_idx, ac_data_idx[idx1])
    }
}

##############################
########### Output ###########
##############################
# [1] "0% complete"
# [1] "10% complete"
# [1] "20% complete"
# [1] "30% complete"
# [1] "40% complete"
# [1] "50% complete"
# [1] "60% complete"
# [1] "70% complete"
# [1] "80% complete"
# [1] "90% complete"
# [1] "100% complete"

In [ ]:
print(length(false_idx))
if(length(false_idx) > 0) {
  ac_data$touchdown[false_idx] <- F
}

##############################
########### Output ###########
##############################
# [1] 224

In [ ]:
touchdown_df <- ac_data[ac_data$touchdown, c("aircraft", "ts")]
rows <- nrow(touchdown_df) + 1

while(rows != nrow(touchdown_df)) {
    rows <- nrow(touchdown_df)
    print(rows)
    touchdown_df[, "next_touchdown_ts" := c(ts[-1], NA), by = "aircraft"]
    touchdown_df$touchdown_time_diff <- touchdown_df$next_touchdown_ts - touchdown_df$ts
    touchdown_df$touchdown_time_diff[is.na(touchdown_df$touchdown_time_diff)] <- 600
    touchdown_df <- touchdown_df[touchdown_df$touchdown_time_diff >= 600, ]
}
touchdown_df$touchdown <- T
ac_data$touchdown <- NULL
ac_data <- merge(ac_data, touchdown_df, all.x = T, all.y = T)
ac_data$touchdown[is.na(ac_data$touchdown)] <- F
sum(ac_data$touchdown)

##############################
########### Output ###########
##############################
# [1] 5077
# [1] 5077

# Testing definition of touchdown

In [ ]:
identify_airport <- function(lat, lon, lat_half_range = airport_lat_half_range,
                             lon_half_range = airport_lon_half_range) {
  df <- airports[data.table::between(airports$lat, lower = lat - lat_half_range,
                                     upper = lat + lat_half_range) &
                   data.table::between(airports$lon, lower = lon - lon_half_range,
                                       upper = lon + lon_half_range), ]
  if(nrow(df) > 1) {
    dists <- apply(df, 1, function(row) {
      row["lat"] <- as.numeric(row["lat"])
      row["lon"] <- as.numeric(row["lon"])
      return(distm(x = c(row["lon"], row["lat"]), y = c(lon, lat)))
    })
    df <- df[which.min(dists), ]
    df$dist <- min(dists)
  } else if(nrow(df) == 1) {
    df$dist <- 0
  } else {
    dists <- apply(airports, 1, function(row) {
      row["lat"] <- as.numeric(row["lat"])
      row["lon"] <- as.numeric(row["lon"])
      return(distm(x = c(lon, lat), y = c(row["lon"], row["lat"])))
    })
    df <- airports[which.min(dists), ]
    df$dist <- min(dists)
  }
  return(df[, c("airport", "dist")])
}


touchdown <- ac_data[ac_data$touchdown, ]
landed_airport <- list()

for(i in 1:nrow(touchdown)) {
  landed_airport[[i]] <- identify_airport(touchdown$lat[i], touchdown$lon[i])
}
all(sapply(landed_airport, nrow) == 1)
condition <- sapply(landed_airport, function(row) row$dist) <= 10000
print(mean(condition)) # landed location is within 10 km of airport
touchdown_idx <- which(ac_data$touchdown)
non_touchdown <- touchdown_idx[!condition]
ac_data$touchdown[non_touchdown] <- F
ac_data$airport <- ""
ac_data$airport[ac_data$touchdown] <- sapply(landed_airport, function(row) row$airport)[condition]

##############################
########### Output ###########
##############################
# [1] TRUE
# [1] 0.8733504

In [ ]:
precision_recall <- T
if(precision_recall) {
  dests <- ac_data$last_destination[ac_data$touchdown & ac_data$last_destination != ""]
  print(mean(dests == ac_data$airport[ac_data$touchdown & ac_data$last_destination != ""]))
  # Accuracy of 'given' destination: 0.9343
  # Actually the precision is 0.9635 (from azimuth analysis)
  
  # JFK recall (assuming 'airport' is true value)
  tbl <- table(ac_data$destination[ac_data$touchdown & ac_data$airport == "JFK"])
  print(sort(tbl, decreasing = T)/sum(tbl))
  # JFK recall is 0.9424, "" is the main issue. In some cases the aircraft destination was different?
}

##############################
########### Output ###########
##############################
# [1] 0.92164

#         JFK                     BNA         DCA         TPA 
# 0.982824427 0.011450382 0.001908397 0.001908397 0.001908397

In [ ]:
lat_lon_boundary_flag <- function(lt, ln, last_lt, last_ln,
                                  lower_lt = lower_lat, lower_ln = lower_lon, upper_lt = upper_lat,
                                  upper_ln = upper_lon, lt_range = lat_range, ln_range = lon_range) {
  ans <- (ln < ((lower_ln + ln_range) | last_ln < (lower_ln + ln_range))) |
    ((ln > (upper_ln - ln_range) | last_ln > (upper_ln - ln_range))) |
    ((lt < (lower_lt + lt_range) | last_ln < (lower_lt + lt_range))) |
    ((lt > (upper_lt - lt_range) | last_ln > (upper_lt - lt_range)))
  return(ans)
}


ac_data$landed <- ac_data$touchdown
#ac_data$entered <- (ac_data$V2 != ac_data$last_V2) |
#  (lat_lon_boundary_flag(ac_data$lat, ac_data$lon, ac_data$last_lat, ac_data$last_lon) &
#          ac_data$last_ts_diff > 1080)
# Aircraft 'entered' if 
# - last position was close to the boundary
# - current position is close to the boundary
# - time between last sample and current sample > 18 mins (~ min turnaround time after landing)
ac_data$entered <- lat_lon_boundary_flag(ac_data$lat, ac_data$lon, ac_data$last_lat, ac_data$last_lon) & ac_data$last_ts_diff > 1080

ac_data$entered[is.na(ac_data$entered)] <- F
ac_data$expired <- (ac_data$last_ts_diff > ts_diff_expired)
ac_data$landed_or_entered_or_expired <- ac_data$landed | ac_data$entered | ac_data$expired
ac_data[, "num_id" := cumsum(landed_or_entered_or_expired), by = "aircraft"]
table(ac_data$num_id)
ac_data$id <- paste0(ac_data$aircraft, "_", ac_data$num_id)

##############################
########### Output ###########
##############################
#       0       1       2       3       4       5       6       7       8       9 
# 1826566  648768  931478  504327  291799  283883  188793  114659   93193   55825 
#      10      11      12      13      14      15      16      17      18      19 
#   32350   20578   13246    8733    2847     704     635      58     906      32

# Hand-picking numeric variables to reduce floating precision

In [ ]:
numerics <- colnames(ac_data)[sapply(ac_data, class) == 'numeric']
numerics

##############################
########### Output ###########
##############################
#  [1] "V1"                        "lat"                      
#  [3] "lon"                       "altitude"                 
#  [5] "ground_flag"               "quadrant"                 
#  [7] "last_lat"                  "last_lon"                 
#  [9] "last_altitude"             "last_ground_flag"         
# [11] "last_quadrant"             "next_lat"                 
# [13] "next_lon"                  "next_altitude"            
# [15] "next_ground_flag"          "next_quadrant"            
# [17] "max_ac_alt"                "est_climb_rate"           
# [19] "climb_rate_ratio"          "max_altitude_touchdown_id"
# [21] "touchdown_time_diff"

Unfortunately float::fl doesn't work as expected as it just stores an 4 byte integer that's ready for converesion back to 8 byte float

In [ ]:
# ac_data$lat <- float::fl(ac_data$lat)
# ac_data$lon <- float::fl(ac_data$lon)
ac_data$altitude <- as.integer(ac_data$altitude)
ac_data$ground_flag <- ac_data$ground_flag==1
ac_data$quadrant <- as.integer(ac_data$quadrant)
# ac_data$last_lat <- float::fl(ac_data$last_lat)
# ac_data$last_lon <- float::fl(ac_data$last_lon)
ac_data$last_altitude <- as.integer(ac_data$last_altitude)
ac_data$last_ground_flag <- ac_data$last_ground_flag==1
ac_data$last_quadrant <- as.integer(ac_data$last_quadrant)
# ac_data$next_lat <- float::fl(ac_data$next_lat)
# ac_data$next_lon <- float::fl(ac_data$next_lon)
ac_data$next_altitude <- as.integer(ac_data$next_altitude)
ac_data$next_ground_flag <- ac_data$next_ground_flag==1
ac_data$next_quadrant <- as.integer(ac_data$next_quadrant)
ac_data$max_ac_alt <- as.integer(ac_data$max_ac_alt)
# ac_data$est_climb_rate <- float::fl(ac_data$est_climb_rate)
# ac_data$climb_rate_ratio <- float::fl(ac_data$climb_rate_ratio)
ac_data$max_altitude_touchdown_id <- as.integer(ac_data$max_altitude_touchdown_id)

# Altitude ratio filter

In [ ]:
ac_data[, c("max_alt_ac_id", "q99_alt_ac_id") := list(max(altitude), quantile(altitude, 0.99)), by = id]
ac_data$max_99_ratio <- ac_data$max_alt_ac_id / ac_data$q99_alt_ac_id
ac_data <- ac_data[ac_data$max_99_ratio < alt_100_99_perc_ratio_max, ] # Remove only the sample, not the entire aircraft's data
# Also ends up removing IDs with altitude=0 throughout, which is okay if we don't want to bother about taxiing after touchdown
# Otherwise taxiing should be preprocessed differently - somehow don't create a new ID after touchdown till the aircraft becomes inactive
ac_data <- ac_data[ac_data$max_alt_ac_id <= altitude_ceiling & ac_data$max_alt_ac_id > 0, ] # Explicitly removing aircraft that don't get off the ground
# This also removes taxiing phase
rownames(ac_data) <- NULL

# Saving final prepropcessed data

In [ ]:
setDT(ac_data)
setorder(ac_data, aircraft, ts)
write_feather(ac_data[, c("aircraft", "id", "ts", "ts_readable", "lon", "lat", "altitude", "ground_speed", "azimuth", "source", "destination")], "ac_data_processed.feather")
ac_data[, c("last_touchdown", "next_touchdown", "last_entered", "next_entered", "last_airport", "next_airport") :=
          list(c(F, touchdown[-.N]), c(touchdown[-1], F), c(F, entered[-.N]), c(entered[-1], F), c("", airport[-.N]), c(airport[-1], "")), by = "aircraft"]
ac_data$next_touchdown[is.na(ac_data$next_touchdown)] <- F

In [ ]:
jfk_data <- data.frame(id = unique(ac_data$id[ac_data$next_airport == "JFK" & ac_data$next_touchdown &
                                              ac_data$source != "JFK"]), jfk_landing_flag = T)
jfk_data <- merge(jfk_data, ac_data, all.x = T, all.y = T)
jfk_data$jfk_landing_flag[is.na(jfk_data$jfk_landing_flag)] <- F
setDT(jfk_data)
jfk_data[, "max_id_alt" := max(altitude), by = "id"]
write.csv(jfk_data[, c("id", "aircraft", "lat", "lon", "ts", "altitude", "climb_rate",
                       "azimuth", "ground_speed", "jfk_landing_flag", "source",
                       "destination", "ground_flag")],
          "samples_with_jfk_landing_flag.csv", row.names = F)
write_feather(ac_data, "full_ac_data.feather")
# saveRDS(jfk_data[jfk_data$max_id_alt >= 20000, ], "high_ac_data.Rds")
write_feather(jfk_data, "full_jfk_data.feather")

In [ ]:
length(unique(jfk_data$id))

##############################
########### Output ###########
##############################
# [1] 20486